In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pyflann import *
import datetime

In [2]:
# 将工作环境调整至/data/planogram下
path = os.path.abspath("..")
path = path.replace('\\', '/')
os.chdir(path + "/data")

## 读取待检测的目标图片与模板

In [6]:
# 目标图片
target = cv2.imread("planogram/IMG_0114.jpg", 1)
template_list = ['template/template.png']
template_name = ['洁云']
template_dict = {}
for template, category in zip(template_list, template_name):
    template_dict[category] = cv2.imread(template, 1)

## image_show函数：用matplotlib库展示图片
- 可自动识别彩色图像和灰度化图像

In [12]:
def image_show(img: np.ndarray):
    # 彩色RGB图像
    if img.ndim == 3:
        plt.figure(figsize=(10, 12))
        # opencv以BGR模式存储图片，如果要使用plt.imshow()函数显示图像
        # 需要将图片转换为RGB模式
        image_show = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(image_show, cmap='gray', interpolation='bicubic')
        plt.xticks([]), plt.yticks([])
        plt.show()
    # 灰度图像    
    elif img.ndim == 2:
        plt.figure(figsize=(10, 12))
        plt.imshow(img, cmap='gray', interpolation='bicubic')
        plt.xticks([]), plt.yticks([])
        plt.show()
    else:
        return None

<b><font size=5 font color=#DC143C>旋转图像模块待完成</font></b>

In [ ]:
def image_horizontal_segmentation(image: np.ndarray) -> list:
    # 首先对图像进行灰度化
    # 灰度化公式(各通道权重大小) Y = 0.299R + 0.587G + 0.114B
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 使用高斯滤波器对图像进行平滑以便于使用canny算子检测图像边缘
    # 滤波器尺寸为5*5, 高斯函数标准差为2
    blur_image = cv2.GaussianBlur(gray_image, (5, 5), 2)
    # 使用canny算法检测图像边缘
    # 低阈值设置50, 高阈值设置200
    canny_image = cv2.Canny(blur_image, 50, 200)
    # 使用霍夫变换检测应用了canny算法的边缘图像中的直线
    # 第一个参数代表用于搜索距离参数空间的精度, 第二个参数代表用于搜索角度参数空间的精度
    # 最后一个参数代表一条直线上的最小像素点数量
    lines = cv2.HoughLines(canny_image, 1, np.pi/180, 350)
    lines_array = []
    for line in lines:
        lines_array.append(line[0])
    
    # 将直线的角度参数和距离参数保存到dataframe中
    lines_data = pd.DataFrame(np.array(lines_array), columns=['rho', 'theta'])
    # 将直线的角度从弧度转换为角度
    lines_data['angle'] = lines_data['theta'].map(lambda x: round(np.rad2deg(x)))
    # 取直线角度的众数作为图片的角度(用于检测货架是否水平)
    image_angle = lines_data['angle'].value_counts().index[0]
    if np.abs(image_angle - 90) <= 1:
        print("图像货架基本水平.")
    else:
        print("图像需要进行旋转.")
    
    # 计算canny边缘图像每行上像素点的数量和, 利用货架上像素点比较密集和变化剧烈的特征
    # 利用灰度水平直方图的一阶差分对图像进行水平分割
    gray_sum = []
    for line in canny_image:
        gray_sum.append(line.sum())
    
    gray_sum = np.array(gray_sum)
    gray_df = pd.DataFrame(gray_sum, columns=['gray_sum'])
    # 归一化
    gray_df_2 = gray_df/gray_sum.max()
    gray_df_3 = gray_df_2.diff().apply(np.abs)
    for i in gray_df_3[gray_df_3['gray_sum'] >= (gray_df_3['gray_sum'].max() * 0.6)].index:
        
    
    